In [ ]:
import os
from crewai import Agent, Process
from langchain.tools import tool
from langchain_google_genai import GoogleGenerativeAI
from textwrap import dedent
from crewai import Crew, Task
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Model
llm = GoogleGenerativeAI(model="gemini-pro")

In [ ]:
class FileTools:
    @tool("Write File with content")
    def write_file(data: str):
        """Useful to write a file to a given path with a given content. 
        The input to this tool should be a pipe (|) separated text 
        of length two, representing the full path of the file, 
        including the ./lore/, and the written content you want to write to it.
        """
        try:
            path, content = data.split("|")
            path = path.replace("\n", "").replace(" ", "").replace("`", "")
            if not path.startswith("./lore"):
                path = f"./lore/{path}"
            with open(path, "w") as f:
                f.write(content)
            return f"File written to {path}."
        except Exception:
            return "Error with the input format for the tool."

In [ ]:
idea_analyst = Agent(
    role="Idea Analyst",
    goal="Comprehensively analyse an idea to prepare blueprints for the article to be written",
    backstory="""You are an experienced content analyst, well versed in analyzing 
    an idea and preparing a blueprint for it.""",
    llm=llm,
    verbose=True
)
writer = Agent(
    role="Fiction Writer",
    goal="Write compelling fantasy and sci-fi fictions from the ideas given by the analyst",
    backstory="""A renowned fiction-writer with 2 times NYT 
    a best-selling author in the fiction and sci-fi category.""",
    llm=llm,
    verbose=True
)

editor = Agent(
    role="Content Editor",
    goal="Edit contents written by writer",
    backstory="""You are an experienced  editor with years of 
    experience in editing books and stories.""",
    llm=llm,
    tools=[FileTools.write_file],
    verbose=True
)

In [ ]:
class CreateTasks:

    def expand_idea():
        return dedent(""" Analyse the given task {idea}. Prepare comprehensive pin-points
                for accomplishing the given task.
                Make sure the ideas are to the point, coherent, and compelling.
                Make sure you abide by the rules. Don't use any tools.
                
                RULES:
                - Write ideas in bullet points.
                - Avoid adult ideas.
            """)

    def write():
        return dedent("""Write a compelling story in 1200 words based on the blueprint 
        ideas given by the Idea 
              analyst.
              Make sure the contents are coherent, easily communicable, and captivating.
               Don't use any tools.

              Make sure you abide by the rules.

              RULES:
              - Writing must be grammatically correct.
              - Use as little jargon as possible

              """)

    def edit():
        return dedent("""
    Look for any grammatical mistakes, edit, and format if needed.
    Add title and subtitles to the text when needed.
    Do not shorten the content or add comments.
    Create a suitable filename for the content with the .txt extension.
    You MUST use the tool to save it to the path ./lore/(your title.txt).
            """)

In [ ]:
class ContentWritingCrew():
    def __init__(self, idea):
        self.idea = idea

    def __call__(self):
        tasks = self._create_tasks()
        crew = Crew(
            tasks=[tasks],
            agents=[idea_analyst, writer, editor],
            verbose=True,
            process=Process.sequential 
        )
        result = crew.kickoff({"topic": self.idea})
        return result

    def _create_tasks(self):
        idea = CreateTasks.expand_idea().format(idea=self.idea)
        expand_idea_task = Task(
            description=idea,
            agent=idea_analyst,
            expected_output='A comprehensive 5 paragraphs long report on the latest AI trends.'
        )
        write_task = Task(
            description=CreateTasks.write(),
            agent=writer,
            expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
        )
        edit_task = Task(
            description=CreateTasks.edit(),
            agent=editor,
            expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.'
        )
        return [expand_idea_task, write_task, edit_task]

In [ ]:
if __name__ == "__main__":
    dir = "./lore"
    if not os.path.exists(dir):
        os.mkdir(dir)
    idea = input("idea: ")
    my_crew = ContentWritingCrew(idea=idea)
    result = my_crew()
    print(dedent(result))